#Connecting to the blob storage, fetching the csv files, creating dataframes and temporary views for analysis

In [0]:
storage_account_name = 'inputstorageag'

storage_account_access_key = 'xxxxxx'

spark.conf.set('fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net', storage_account_access_key)

blob_container = 'landing'



In [0]:
filePath = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net/Advertisers.csv"

AdvertisersDf = spark.read.format('csv').load(filePath, inferSchema = True, header = True)

In [0]:
filePath = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net/Affiliate_Network.csv"
Affiliate_NetworkDf = spark.read.format('csv').load(filePath, inferSchema = True, header = True)


In [0]:
filePath = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net/Clicks.csv"
ClicksDf = spark.read.format('csv').load(filePath, inferSchema = True, header = True)

In [0]:
filePath = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net/CouponCodes.csv"
CouponCodesDf = spark.read.format('csv').load(filePath, inferSchema = True, header = True)

In [0]:
filePath = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net/DomainNames.csv"
DomainNamesDf = spark.read.format('csv').load(filePath, inferSchema = True, header = True)

In [0]:
filePath = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net/Orders.csv"
OrdersDf = spark.read.format('csv').load(filePath, inferSchema = True, header = True)

In [0]:
filePath = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net/PageViews.csv"
PageViewsDf = spark.read.format('csv').load(filePath, inferSchema = True, header = True)

In [0]:
filePath = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net/PartnerWebsites.csv"
PartnerWebsitesDf = spark.read.format('csv').load(filePath, inferSchema = True, header = True)

In [0]:
filePath = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net/Promotions.csv"
PromotionsDf = spark.read.format('csv').load(filePath, inferSchema = True, header = True)

In [0]:
AdvertisersDf.createOrReplaceTempView("Advertisers")

In [0]:
Affiliate_NetworkDf.createOrReplaceTempView("Affiliate_Network")

In [0]:
ClicksDf.createOrReplaceTempView("Clicks")

In [0]:
CouponCodesDf.createOrReplaceTempView("CouponCodes")

In [0]:
DomainNamesDf.createOrReplaceTempView("DomainNames")

In [0]:
OrdersDf.createOrReplaceTempView("Orders")

In [0]:
PageViewsDf.createOrReplaceTempView("PageViews")

In [0]:
PartnerWebsitesDf.createOrReplaceTempView("PartnerWebsites")

In [0]:
PromotionsDf.createOrReplaceTempView("Promotions")

#Tasks from case study

Top 3 retailers/advertisers by revenue

In [0]:
%sql
--ranking advertisers by revenue (revenue understood as sum of all commision amount)
WITH t1 AS (
  SELECT o.*, 
  DENSE_RANK() OVER(ORDER BY Revenue DESC) AS ranking
  FROM (SELECT o.*,
        SUM(COMMISSIONAMOUNT) OVER (PARTITION BY ADVERTISERID) AS Revenue
        FROM Orders o) o
)

--choosing top 3 advertisers with their names 
SELECT DISTINCT t1.ADVERTISERID, a.ADVERTISERNAME
FROM t1
LEFT JOIN Advertisers a ON a.ADVERTISERID=t1.ADVERTISERID
WHERE ranking<=3

/*Additional comments to this task: 
1. List of adversisers in incompleted. Not all AdvertiserId's that are in Orders table are present in Advertisers table. 
*/


ADVERTISERID,ADVERTISERNAME
6845,null
80194,null
7231,null


Top 3 retailers/advertisers by page views

In [0]:
%sql
--calculating number of page views per domain
WITH t1 AS(
SELECT DOMAIN_NAME, COUNT(DISTINCT EVENT_ID) AS PageView
FROM PageViews
GROUP BY 1),

--ranking domains by number of page views
t2 AS(
SELECT DOMAIN_NAME, RANK() OVER( ORDER BY PageView DESC) AS ranking
FROM t1)

--identifying top 3 domains
SELECT * FROM t2 WHERE ranking <=3

/*Additional comments to this task: 
1. The description of task states that I should identify top 3 advertisers by number of page views. I wasn't sure whether domain is the same thing as advertiser (though they have similar and matching names)
but otherwise it is impossible to perform this task (it is impossible to link PageViews table with Advertisers table due to missing key in Orders table - DomainNameID)
2. Even if there was a DomainNameID key in the Orders table I think that we still wouldn't get correct data as we would end up counting ONLY these Page Views that led to Orders and I'm sure you loose a part of users at each stage of the funnel (PageView -> Click -> Order). In order to get a AdvertiserName there should be a foreign key (advertiserID) in the PageViews table.
*/

DOMAIN_NAME,ranking
ubereats.com,1
amazon.com,2
wayfair.com,3


Average Commission Rate (Commission Rate = commissions/sale amount)

In [0]:
%sql
SELECT ROUND(SUM(COMMISSIONAMOUNT)/SUM(SALEAMOUNT), 5) AS CommisionRate 
FROM Orders

/* Additional comments: 
1. I was thinking of using AVG(COMMISSIONAMOUNT/SALEAMOUNT) but in this case I wouldn't take weights of Sale Amount into consideration and as a consequence I wouldn't differentiate between them.
*/

CommisionRate
0.09982


Number of unique page views

In [0]:
%sql
SELECT COUNT(DISTINCT PV.EVENT_ID) AS UniquePageViews
FROM PageViews AS PV

/* Additional comments: 
My concern here was the methodology of assigning EVENT_IDs. Are there changing after every time a given user refreshes the website? Or are there some time criteria after which there is a new event_id for the same user (30 minutes of inactivity etc.)
*/

UniquePageViews
17792901


Rolling sum of order count

In [0]:
%sql
--first calculate the number of orders per day
WITH t1 AS(
SELECT  CAST(TRANSACTIONDATE AS date) AS TransDate, COUNT(DISTINCT ORDERID) AS NoOrders
FROM Orders
GROUP BY 1
ORDER BY 1 ASC
)
--calcualting rolling sum 
SELECT TransDate, NoOrders, SUM(NoOrders) OVER (ORDER BY TransDate ASC) AS RollingSum
FROM t1


TransDate,NoOrders,RollingSum
2022-02-07,16304,16304
2022-02-08,16277,32581
2022-02-09,15613,48194
2022-02-10,15182,63376
2022-02-11,15772,79148
2022-02-12,14347,93495
2022-02-13,14481,107976
2022-02-14,14756,122732
2022-02-15,15759,138491
2022-02-16,15764,154255


Month-over-month comparison of revenue broken down by affiliate network (is revenue going up or down for the various affiliate networks?)

In [0]:
%sql
WITH t1 AS (SELECT 
AFFILIATENETWORKNAME
, February
, March
FROM 
(
  --calculating daily average nnumber of orders per month for all of affiliate networks
SELECT SUM(COMMISSIONAMOUNT)/COUNT(DISTINCT CAST(TRANSACTIONDATE AS Date)) AS MonthlyRev, 
CASE WHEN MONTH(TRANSACTIONDATE) =2 THEN 'February' ELSE 'March' END AS MonthName, 
AF.AFFILIATENETWORKNAME
FROM Orders AS O
LEFT JOIN Affiliate_Network AS AF ON AF.AFFILIATENETWORKID = O.AFFILIATENETWORKID
GROUP BY 2, 3
) AS t
--creating a pivot table to compare between months
PIVOT
(
  SUM(MonthlyRev)
  FOR MonthName IN ('February', 'March' )
)
)
SELECT *,
CASE WHEN March>February THEN 'increase' ELSE 'decrease' END AS month_over_month
FROM t1

/* additional comments: 
1. Data that you provided contains different number of days for February and March
(less for March). In order to be able to compare between those two months I calculated daily averages for each month. Otherwise we'd probably see a decreasing trend in March for most of the afiliate networks. 
2. Taking the above mentioned approach doesn't have to be 100% correct as we need to make sure that we take seasonality factors into consideration. I assume that as a web platform you generate more traffic (which results in higher number of orders) during weekends so it should be taken into consideration in month-over-month comparison. In this case and data that you sent we should make sure that there is a same share of weekends in both of the analyzed months.
*/

AFFILIATENETWORKNAME,February,March,month_over_month
BrandReward,4585.966078039035,2378.237348744955,decrease
Impact Radius,14531.881054124933,12113.970228231588,decrease
DigiDip,9334.76751281391,10420.32845725199,increase
LinkShare,5286.843448826907,4766.20981436999,decrease
FlexOffers,70.2489232200872,98.35615687117479,increase
LinkConnector,105.40143645947228,110.3543737845956,increase
AffiliateWindow,1621.570894693619,1423.2585258341644,decrease
AvantLink,827.8595722989961,826.1647920487161,decrease
ShareASale,1947.3662617584132,1925.9428237489733,decrease
Partnerize,2094.799255660372,2026.1716084289085,decrease


#Connection to database and saving the data from files into SQL tables

In [0]:
connection = "jdbc:sqlserver://agserverexcercice.database.windows.net:1433;database=xxxxx;user=xxxxxxx;password=xxxxxxprojecct;encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;" 

In [0]:
AdvertisersDf.write.jdbc(url=connection, table='dbo.Advertisers')

In [0]:
Affiliate_NetworkDf.write.jdbc(url=connection, table ='dbo.AffiliateNetwork')

In [0]:
Clicks = ClicksDf.limit(1000)

In [0]:
Clicks.write.jdbc(url=connection, table ='dbo.Clicks')


In [0]:
CouponCodesDf.write.jdbc(url=connection, table ='dbo.CouponCodes')

In [0]:
DomainNamesDf.write.jdbc(url=connection, table ='dbo.DomainNames')

In [0]:
OrdersDf.write.jdbc(url=connection, table ='dbo.Orders')

In [0]:
PageViews = PageViewsDf.limit(1000)

In [0]:
PageViews.write.jdbc(url=connection, table ='dbo.PageViews')

In [0]:
PartnerWebsitesDf.write.jdbc(url=connection, table ='dbo.PartnerWebsites')

In [0]:
PromotionsDf.write.jdbc(url=connection, table ='dbo.Promotions')
